# 3D backends

On top of the `K3DBackend` previously introduced, we can also create 3D plots using `MayaviBackend` or `PlotlyBackend`.

In [1]:
%matplotlib widget
from sympy import *
from sympy.plotting.plot import plot3d, plot3d_parametric_surface, plot3d_parametric_line
from spb.plotly import PB
from spb.mayavi import MB
from spb.k3d import KB
import numpy as np
var("u, v, x, y")

(u, v, x, y)

Let's start with a simple example which illustrate that no matter what backend we are using, we ultimately have to decide if the plot is correct or not:

In [ ]:
expr = re(atan(x + I*y))
n = 200
plot3d(expr, backend=KB, nb_of_points_x=n, nb_of_points_y=n)

Here, the vertical wall at $x=0$ is rendered, but should not!!! There is a discontinuity there.

As we can see above, by default a color map is applied to the surface, ranging from the minimum and maximum value of the z-coordinate. If we wish to switch to an automatic solid coloring, we need to set the `use_cm=False`, for example:

In [3]:
n = 500
expr = cos(2 * pi * x * y)
plot3d(expr, (x, -2, 2), (y, -2, 2), backend=KB,
       nb_of_points_x=n, nb_of_points_y=n, use_cm=False)

Notebook initialized with ipy backend.
Notebook initialized with ipy backend.


Output()

In [4]:
n = 100
expr = (
    2 * (1 - exp(u / (6 * pi))) * cos(u) * cos(v / 2)**2,
    2 * (-1 + exp(u / (6 * pi))) * sin(u) * cos(v / 2)**2,
    1 - exp(u / (3 * pi)) - sin(v) + exp(u / (6 * pi)) * sin(v)
)
plot3d_parametric_surface(*expr, (u, 0, 6*pi), (v, 0, 2*pi),
                          backend=KB, nb_of_points_u=n, nb_of_points_v=n, use_cm=True)

Notebook initialized with ipy backend.
Notebook initialized with ipy backend.


Output()

Let's now try the `MayaviBackend`, which provides somewhat a similar plotting experience of `K3DBackend`, at least in terms of performance:

In [ ]:
n = 100
r = 3
expr = (
    (r + cos(u / 2) * sin(v) - sin(u / 2) * sin(2 * v)) * cos(u),
    (r + cos(u / 2) * sin(v) - sin(u / 2) * sin(2 * v)) * sin(u),
    sin(u / 2) * sin(v) + cos(u / 2) * sin(2 * v)
)
plot3d_parametric_surface(*expr, (u, 0, 2**pi), (v, 0, 2*pi),
                          backend=MB, nb_of_points_u=n, nb_of_points_v=n)

To interact with the plot:
* Left Click + Drag: rotate the plot.
* Mouse wheel click + Drag up/down: zoom in / zoom out.
* Shift + Left Click + Drag: pan the plot.

As we can see, Mayavi doesn't provide reference grids, but it provides bounding boxes. As with all other backends, to hide the grids or the bounding boxes we set `axis=False`.

Another difference in comparison to `K3DBackend` is that the plot window doesn't use all the available width, hence we might need to set the `size=(width, height)` option.

Let's now try `PlotlyBackend`. The main difference between `PlotlyBackend` and `K3DBackend`/`MayaviBackend` is that the former can stretch the axis, whereas the latter (being more engineering-oriented) uses a fixed aspect ratio representing reality. We can control this behaviour by setting an appropriate value for the `aspect_ratio` keyword:

In [ ]:
plot3d_parametric_surface(*expr, (u, 0, 2**pi), (v, 0, 2*pi),
                          backend=PB, nb_of_points_u=n, nb_of_points_v=n, aspect_ratio="cube")

Note that the z axis is stretched!

The two other major differences are:
* `PlotlyBackend` is consistently slower at rendering 3D objects than the other two backends.
* `PlotlyBackend` doesn't natively support wireframe. While it is possible to draw wireframe by setting `wireframe=True`, it will significantly slow down the rendering process, even for a small number of discretization points. Therefore, it is advisable not to use wireframes with Plotly.
* By moving the cursor over the surface, we can actually see the coordinates of the "selected" point. This is not currently possible with `K3DBackend` or `MayaviBackend`.

In [ ]:
n = 100
expr = (
    (-(2/15) * cos(u) * (3 * cos(v) - 30 * sin(u) + 90 * cos(u)**4 * sin(u) -
    60 * cos(u)**6 * sin(u) + 5 * cos(u) * cos(v) * sin(u))),
    (-(1/15) * sin(u) * (3 * cos(v) - 3 * cos(u)**2 * cos(v) - 48 * cos(u)**4 * cos(v) +
    48 * cos(u)**6 * cos(v) - 60 * sin(u) + 5 * cos(u) * cos(v) * sin(u)
    - 5 * cos(u)**3 * cos(v) * sin(u) - 80 * cos(u)**5 * cos(v) * sin(u) +
    80 * cos(u)**7 * cos(v) * sin(u))),
    (2/15) * (3 + 5 * cos(u) * sin(u)) * sin(v)
)
plot3d_parametric_surface(*expr, (u, 0, pi), (v, 0, 2 * pi),
                          backend=KB, nb_of_points_u=n, nb_of_points_v=n)

In [ ]:
# https://mathematica.stackexchange.com/a/37715
expr = (
    cos(v) * (6 - (5/4 + sin(3 * u)) * sin(u - 3 * v)), 
     (6 - (5/4 + sin(3 * u)) * sin(u - 3 * v)) * sin(v), 
     -cos(u - 3 * v) * (5/4 + sin(3 * u))
)
plot3d_parametric_surface(*expr, (u, 0, 2 * pi), (v, 0, 2 * pi),
                          backend=KB, nb_of_points_x=u, nb_of_points_v=n)

In [6]:
import k3d
u = np.linspace(-5,5,50)
v = np.linspace(-5,5,50)
u,v = np.meshgrid(u,v)
xx,yy,zz = np.cos(u + v), np.sin(u - v), u - v, 
#xx, yy, zz = s2.get_meshes()
rows, cols  = xx.shape
def ij2k(i,j):
    return  cols*i+j 

indices = []
for i in range(1,rows):
    for j in range(1,cols):
        indices.append( [ij2k(i,j),ij2k(i-1,j),ij2k(i,j-1)] )
        indices.append( [ij2k(i-1,j-1),ij2k(i,j-1),ij2k(i-1,j)] )

plot = k3d.plot()

plt_mesh = k3d.mesh(np.stack([xx.flatten(),yy.flatten(),zz.flatten()]).T,\
   indices,\
   side='double',\
   attribute=zz.flatten())
plot += plt_mesh
plot.display()

Output()

In [11]:
plot3d_parametric_surface(cos(u + v), sin(u - v), u - v, (u, 0, pi / 2), (v, 0, 4*pi), backend=KB, wireframe=True)

Notebook initialized with ipy backend.
Notebook initialized with ipy backend.


Output()

In [2]:
r = 2 + sin(7 * u + 5 * v)
expr = (
    r * cos(u) * sin(v),
    r * sin(u) * sin(v),
    r * cos(v)
)
n = 1000
plot3d_parametric_surface(*expr, (u, 0, 2 * pi), (v, 0, pi),
                          backend=KB, nb_of_points_u=n, nb_of_points_v=n, show_label=True)

Notebook initialized with ipy backend.
Notebook initialized with ipy backend.


Output()